In [7]:
import numpy as np
import pandas as pd
import os
from sklearn import metrics
from scipy.stats import zscore
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import cohen_kappa_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error, roc_curve, classification_report,auc)
import keras
import tensorflow as tf
df = pd.read_csv(r"C:\Users\Admin\Documents\ALIYA\1. Aliya Thesis\DATASET\Bluetooth\MergedFinal1.csv", header=None)
print("Read {} rows.".format(len(df)))
df.dropna(inplace=True,axis=1) 
df.columns = [
     'Frame_length_stored_into_the_capture_file_per_100msec',
    'Length_per_100msec',
    'L2CAP_count_per_100msec',
    'HCI_ACL_count_per_100msec',
    'HCI_EVT_count_per_100msec',
    'Received_count_per_100msec',
    'Sent_count_per_100msec',   
    'Command_Complete_count_per_100msec',
    'Disconnect_complete_count_per_100msec',
    'outcome'
]

ENCODING = 'utf-8'

def expand_categories(values):
    result = []
    s = values.value_counts()
    t = float(len(values))
    for v in s.index:
        result.append("{}:{}%".format(v,round(100*(s[v]/t),2)))
    return "[{}]".format(",".join(result))
        
def analyze(df):
    print()
    cols = df.columns.values
    total = float(len(df))

    print("{} rows".format(int(total)))
    for col in cols:
        uniques = df[col].unique()
        unique_count = len(uniques)
        if unique_count>100:
            print("** {}:{} ({}%)".format(col,unique_count,int(((unique_count)/total)*100)))
        else:
            print("** {}:{}".format(col,expand_categories(df[col])))
            expand_categories(df[col])
            
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd
    

encode_numeric_zscore(df, 'Frame_length_stored_into_the_capture_file_per_100msec')
encode_numeric_zscore(df, 'Length_per_100msec')
encode_numeric_zscore(df, 'L2CAP_count_per_100msec')
encode_numeric_zscore(df, 'HCI_ACL_count_per_100msec')
encode_numeric_zscore(df, 'HCI_EVT_count_per_100msec')
encode_numeric_zscore(df, 'Received_count_per_100msec')
encode_numeric_zscore(df, 'Sent_count_per_100msec')
encode_numeric_zscore(df, 'Command_Complete_count_per_100msec')
encode_numeric_zscore(df, 'Disconnect_complete_count_per_100msec')

df.groupby('outcome')['outcome'].count()

x_columns = df.columns.drop('outcome')
x = df[x_columns].values
dummies = pd.get_dummies(df['outcome']) # Classification
outcomes = dummies.columns
num_classes = len(outcomes)
y = dummies.values

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from keras import regularizers
from keras.layers.core import Dropout


# Create a test/train split.  25% test
# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=42)


# Create neural net
model = tf.keras.Sequential() # The object is instantiated as a NN of the class Sequential and trained using X_train and y_train.
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu',kernel_regularizer=regularizers.l2(0.0001),))
model.add(Dense(50, input_dim=x.shape[1], kernel_initializer='normal', activation='relu',kernel_regularizer=regularizers.l2(0.0001),))
model.add(Dense(25, input_dim=x.shape[1], kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(0.0001),))
model.add(Dense(15, input_dim=x.shape[1], kernel_initializer='normal', activation='relu',kernel_regularizer=regularizers.l2(0.0001),))
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu',kernel_regularizer=regularizers.l2(0.0001),))
model.add(Dense(1, kernel_initializer='normal', kernel_regularizer=regularizers.l2(0.0001)))
model.add(Dense(y.shape[1],activation='softmax', ))
model.compile(loss='binary_crossentropy', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)
predicted = model.predict(x_test) #Latter the predicted results are stored in the predicted

Read 34163 rows.
Epoch 1/1000
801/801 - 2s - loss: 0.2480 - val_loss: 0.1353
Epoch 2/1000
801/801 - 1s - loss: 0.1122 - val_loss: 0.0844
Epoch 3/1000
801/801 - 1s - loss: 0.0750 - val_loss: 0.0637
Epoch 4/1000
801/801 - 1s - loss: 0.0557 - val_loss: 0.0465
Epoch 5/1000
801/801 - 1s - loss: 0.0383 - val_loss: 0.0361
Epoch 6/1000
801/801 - 1s - loss: 0.0305 - val_loss: 0.0334
Epoch 7/1000
801/801 - 1s - loss: 0.0290 - val_loss: 0.0309
Epoch 8/1000
801/801 - 1s - loss: 0.0273 - val_loss: 0.0313
Epoch 9/1000
801/801 - 2s - loss: 0.0265 - val_loss: 0.0218
Epoch 10/1000
801/801 - 1s - loss: 0.0215 - val_loss: 0.0243
Epoch 11/1000
801/801 - 1s - loss: 0.0193 - val_loss: 0.0197
Epoch 12/1000
801/801 - 1s - loss: 0.0175 - val_loss: 0.0234
Epoch 13/1000
801/801 - 1s - loss: 0.0177 - val_loss: 0.0200
Epoch 14/1000
801/801 - 1s - loss: 0.0170 - val_loss: 0.0165
Epoch 15/1000
801/801 - 1s - loss: 0.0160 - val_loss: 0.0145
Epoch 16/1000
801/801 - 1s - loss: 0.0151 - val_loss: 0.0163
Epoch 17/1000
80

In [1]:
!pip install tflite-model-maker

  Using cached google_auth-1.24.0-py2.py3-none-any.whl (114 kB)
  Attempting uninstall: google-auth
    Found existing installation: google-auth 1.11.2
    Uninstalling google-auth-1.11.2:
      Successfully uninstalled google-auth-1.11.2


In [8]:
#accuracy
predicted = np.argmax(predicted,axis=1)
proba = model.predict_proba(x_test) 
y_eval = np.argmax(y_test,axis=1) 
score = metrics.accuracy_score(y_eval, predicted)
print("Validation score: {}".format(score))

Validation score: 0.9961362838075167


In [11]:
from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import model_spec

import matplotlib.pyplot as plt

In [13]:
# Convert the model

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\Admin\AppData\Local\Temp\tmpewc53kka\assets


In [14]:
# Save the model.
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)
#successfully saved the model in tflite and uploaded on firebase